In [1]:
from matplotlib import pyplot as plt
from importlib import reload
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import random
import scipy
import math


In [2]:
class ResidentialLoad:
    
    # Initializer
    def __init__(self):
        self.csv_path = None
        self.dataframe = pd.DataFrame()
        self.isDH = False
        self.isFlex = False
        self.isNew = False
    
    def description(self):
        return ('Residential Load of type {} with ID {}.'.format(self.__class__.__name__, self.ID))
    
        
    def be_flexible(self, date_list, reduction):
        self.isFlex = True
        self.flexSlack = random.randint(3,9) #hours to spred out consumption
        self.dataframe.index.apply(self.flex(date_list, reduction))

        
    def flex(self, index, date_list, reduction):
        if index.normalize() in date_list and index.hour in ['17','18','19']:
            reduce = reduction * dataframe[index,self.ID]
            dataframe[index] -= reduce
            loc = index.get_loc()
            i_start = - int(index.hour) + 19
            i_end = i_start + self.flexSlack
            reduce /= i_end
            self.dataframe.iloc[loc+i_start:loc+i_end] += reduce
        
    
class HouseNew(ResidentialLoad):
    # Initializer
    def __init__(self, ID):
        self.ID = ID
        self.csv_path = '../data/new_houses.csv'
        self.isDH = False
        self.isFlex = False
        self.isNew = True
        self.dataframe = pd.read_csv(self.csv_path, 
                                     index_col = 0, 
                                     parse_dates = True, 
                                     usecols = [0,random.randrange(1,49,1)],
                                     skiprows = 1,
                                     names = ['Date', self.__class__.__name__ + str(self.ID)])
        
class HouseOld(ResidentialLoad):
    # Initializer
    def __init__(self, ID):
        self.ID = ID
        self.csv_path = '../data/old_houses.csv'
        self.isDH = False
        self.isFlex = False
        self.isNew = False
        self.dataframe = pd.read_csv(self.csv_path, 
                                     index_col = 0, 
                                     parse_dates = True,
                                     usecols = [0, random.randrange(1,35,1)],
                                     skiprows = 1,
                                     names = ['Date', self.__class__.__name__ + str(self.ID)])
        
class HouseDH(ResidentialLoad):
    # Initializer
    def __init__(self, ID):
        self.ID = ID
        self.csv_path = '../data/mixed_ages_houses_district_heating.csv'
        self.isDH = True
        self.isFlex = False
        self.isNew = False
        self.dataframe = pd.read_csv(self.csv_path, 
                                     index_col = 0, 
                                     parse_dates = True, 
                                     usecols = [0,random.randrange(1,37,1)],
                                     skiprows = 1,
                                     names = ['Date', self.__class__.__name__ + str(self.ID)])
        
        
class ApartmentNewDH(ResidentialLoad):
        # Initializer
    def __init__(self, ID):
        self.ID = ID
        self.csv_path = '../data/new_apartments_district_heating.csv'
        self.isDH = False
        self.isFlex = False
        self.isNew = True
        self.dataframe = pd.read_csv(self.csv_path, 
                                     index_col = 0, 
                                     parse_dates = True, 
                                     usecols = [0, random.randrange(1,35,1)],
                                     skiprows = 1,
                                     names = ['Date', self.__class__.__name__ + str(self.ID)])
    

In [ ]:
# Dummy first version of introducing flexibility to load
# curves. The function finds the n coldest day of the year
# and assumes a percentage of loads (default 50%) can be
# flexible. Flexible is interpreted as reducing comsumption
# with a certain percentage (default 0.67). The consumption
# can either be just reduced (default) or also redistributed. 

def introduce_flexibility(dataframe,
                          region,
                          days = 17, 
                          percent_loads = 0.5, 
                          reduction = 0.65, 
                          move_consumption = False):
    
    start_date, end_date = dataframe.index[0], dataframe.index[-1]
    
    days_to_reduce = find_coldest_days(region_path_dict[region], 
                                       num = days, 
                                       start = start_date, 
                                       end = end_date)
    
    dataframe = reduce_loads(dataframe, 
                             percent_loads, 
                             reduction, 
                             days_to_reduce)
    
    if move_consumption:
        print('Please implement this feature')
        
    return dataframe

def reduce_loads(dataframe, percent_loads, reduction, date_list, no_DH = True):
    
    if no_DH: # Only reduce electrical heating
        subframe = dataframe.loc[:,~dataframe.columns.str.contains('DH')] 
        subframe = drop_random_columns(subframe, drop_percent = percent_loads)
        subframe = subframe[subframe.index.hour.isin(['17','18','19'])]
        bool_list = [(x.to_pydatetime().date() in date_list) for x in subframe.index]
        subframe.loc[subframe.index[bool_list], :]
        subframe[subframe.columns] *= (1-reduction)
    
        dataframe.loc[dataframe.index.isin(subframe.index), subframe.columns] = subframe.loc[subframe.index.isin(dataframe.index),subframe.columns].values
    return dataframe

In [ ]:
house2 = HouseNew(ID = 2)

In [ ]:
house2.description()